### Test the Model (Competition)

In [ ]:
%pip install unidecode
%pip install numpy pandas matplotlib seaborn wordcloud
%pip install nltk unidecode
%pip install scikit-learn
%pip install mlxtend
%pip install torch torchvision torchaudio
%pip install transformers
%pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
# import the required packages
# Core
import re
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# NLP
import nltk
from nltk import pos_tag
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords

# Machine Learning - Preprocessing & Modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

# Machine Learning - Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Association Rule Mining
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from unidecode import unidecode
from mlxtend.preprocessing import TransactionEncoder


from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df = pd.read_csv('AA_movie_train_data.csv')

In [6]:
movie_genre = list(df['Genre'].unique())
movie_genre.sort()
movie_genre

['action', 'comedy', 'documentary', 'drama', 'thriller']

In [6]:
genre_mapper = {'action': 0, 'documentary': 1, 'comedy':2, 'drama':3,  'thriller': 4}
df['Genre'] = df['Genre'].map(genre_mapper)
df.head(10)

,Title,Genre,Description
0,Windup (2006),0,Windup is a very original comedy. We follow B...
1,Hitman (2014),0,"Rana and Shuvo, two siblings, are very differ..."
2,Taken by Force (2010),0,When a San Francisco detective goes hunting f...
3,Bui doi Cho Lon (2013),0,"A man name Lam picks up his girlfriend, but i..."
4,Siam Yuth: The Dawn of the Kingdom (2015),0,Thap and Sin are a homeless musician brothers...
5,Ninja Champion (1986),0,A couple camping in the woods is attacked by ...
6,ISIS: Enemies of Humanity (2017),0,ISIS Enemies of Humanity is an upcoming Inter...
7,Veerta (1993),0,Multi-millionaire Balwant Rai is a widower wh...
8,Cherry Bomb (2011),0,In homage to the action films of the early 19...
9,Lai Bhaari (2014),0,Pratapsingh Nimbalkar and his wife Sumitra ar...


In [8]:
df.isna().any()

,0
Title,False
Genre,False
Description,False


In [9]:
df.shape

(5000, 3)

In [10]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode

# Ensure necessary NLTK datasets are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# 🔹 1. Helper functions from SCRIPT 1

# def get_stop_words(stop_file_path, include_spanish=True):
#     with open(stop_file_path, 'r', encoding='utf-8') as f:
#         custom_stopwords = set(line.strip().lower() for line in f if line.strip())

#     if include_spanish:
#         spanish_stopwords = set(nltk_stopwords.words('spanish'))
#         custom_stopwords.update(spanish_stopwords)

#     return frozenset(custom_stopwords)

# def clean_text(text):
#     text = unidecode(text)                     # strip accents
#     text = text.lower()
#     text = re.sub(r"http\S+|www\S+|https\S+", '', text)
#     text = re.sub(r'\S+@\S+', '', text)
#     text = re.sub(r"</?.*?>", " ", text)
#     text = re.sub(r"(\d|\W|_)+", " ", text)
#     text = re.sub(r"\s+", " ", text).strip()
#     return text

# def tokenize_text(text):
#     return word_tokenize(text)

# def get_wordnet_pos(treebank_tag):
#     """Convert TreeBank POS tags to WordNet POS tags"""
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN  # Default to noun

# def lemmatize_with_pos(pos_tagged_tokens):
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []

#     for word, tag in pos_tagged_tokens:
#         pos = get_wordnet_pos(tag)  # You already have this function
#         lemma = lemmatizer.lemmatize(word, pos)
#         lemmas.append(lemma)

#     return lemmas

# # 🔹 2. Full text processing pipeline function from SCRIPT 1
# def full_text_pipeline(text, stopwords):
#     cleaned = clean_text(text)
#     tokens = tokenize_text(cleaned)
#     pos_tags = pos_tag(tokens)
#     lemmas = lemmatize_with_pos(pos_tags)
#     filtered = [lemma for lemma in lemmas if lemma not in stopwords]
#     return filtered, ' '.join(filtered)

# # 🔹 3. Custom stopwords (adjust this with your actual path to stopwords file)
# stopwords = get_stop_words("stopwords.txt", include_spanish=True)

# # 🔹 4. Full processing for dataframe (from SCRIPT 2, with custom stopwords and lemmatization)
# corpus = []

# # Iterate through each row of the DataFrame
# for i in range(0, df.shape[0]):
#     # Get the current text (dialog/script)
#     dialog = df['Description'][i]

#     # Apply full text pipeline (cleaning, tokenization, lemmatization, and stopwords removal)
#     filtered_tokens, filtered_string = full_text_pipeline(dialog, stopwords)

#     # Store the cleaned and processed text
#     corpus.append(filtered_string)

# # Add the processed corpus back to the DataFrame if needed
# df['processed_text'] = corpus

# # Optionally, you can save the results to a CSV
# df[['processed_text']].to_csv('processed_dialog.csv', index=False)

# # This should give you the processed text, cleaned, lemmatized, and stopword-free.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:

df.head()

,Title,Genre,Description
0,Windup (2006),0,Windup is a very original comedy. We follow B...
1,Hitman (2014),0,"Rana and Shuvo, two siblings, are very differ..."
2,Taken by Force (2010),0,When a San Francisco detective goes hunting f...
3,Bui doi Cho Lon (2013),0,"A man name Lam picks up his girlfriend, but i..."
4,Siam Yuth: The Dawn of the Kingdom (2015),0,Thap and Sin are a homeless musician brothers...


In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure 'Genre' column is string
df['Genre'] = df['Genre'].astype(str)
df['Genre'] = df['Genre'].lower()


# Fit on Genre strings
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Genre'])

# 2️⃣ Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 3️⃣ Tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# 4️⃣ Create PyTorch Datasets
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

# 5️⃣ Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.to(device)

# 6️⃣ Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# 7️⃣ Training loop
model.train()
for epoch in range(num_epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_postfix(loss=loss.item())

# 8️⃣ Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# 9️⃣ Metrics
print("\nClassification Report:")
# Convert numerical labels back to their original string representations
target_names = label_encoder.inverse_transform(np.unique(all_labels))
print(classification_report(all_labels, all_preds, target_names=target_names))
print("F1 Score (macro):", f1_score(all_labels, all_preds, average='macro'))

KeyboardInterrupt: 

In [15]:
# Required Libraries
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Encode the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Genre'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Tokenization using RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# PyTorch Dataset
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

# Model initialization
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))
model.to(device)

# Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training loop
model.train()
for epoch in range(num_epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_postfix(loss=loss.item())

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Metrics
print("\nClassification Report:")
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
print("F1 Score (macro):", f1_score(all_labels, all_preds, average='macro'))


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Epoch 2:  77%|███████▋  | 193/250 [02:21<00:41,  1.36it/s, loss=0.127]


KeyboardInterrupt: 

In [ ]:
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))


In [ ]:
# Required Libraries
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Encode the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Genre'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Load ELECTRA tokenizer and model
model_name = "google/electra-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_encoder.classes_)
)
model.to(device)

# Tokenize the text
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# PyTorch Dataset
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training loop
model.train()
for epoch in range(num_epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_postfix(loss=loss.item())

# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Metrics
print("\nClassification Report:")
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
print("F1 Score (macro):", f1_score(all_labels, all_preds, average='macro'))


# Task
Explain why the F1 score changes when run at different times, and then provide a plan for hyperparameter tuning to improve the model's performance, including defining the hyperparameter space, choosing a tuning method, modifying the training code, performing the search, and evaluating the best model.

## Define hyperparameter space

### Subtask:
Decide which hyperparameters you want to tune (e.g., learning rate, batch size, number of epochs) and the range of values to explore for each.


**Reasoning**:
I will define the hyperparameters and their ranges to be used for tuning the model's performance.



In [17]:
# 1. Identify hyperparameters to tune and define their ranges
# Define the hyperparameter space as a dictionary
param_grid = {
    'learning_rate': [1e-5, 3e-5, 5e-5],  # Learning rate for the optimizer
    'per_device_train_batch_size': [8, 16, 32], # Batch size for the DataLoader
    'num_train_epochs': [2, 3, 4] # Number of training epochs
}

print("Hyperparameter tuning space defined:")
print(param_grid)

Hyperparameter tuning space defined:
{'learning_rate': [1e-05, 3e-05, 5e-05], 'per_device_train_batch_size': [8, 16, 32], 'num_train_epochs': [2, 3, 4]}


## Choose tuning method

### Subtask:
Choose a method for hyperparameter search, such as GridSearchCV or RandomizedSearchCV from scikit-learn, or more advanced libraries like Optuna or Hyperopt. GridSearchCV is a good starting point for a smaller search space.


## Modify training code for tuning

### Subtask:
Adapt your existing training and evaluation code to work within the chosen tuning framework. This will likely involve creating a function that takes hyperparameters as input, trains the model, and returns a performance metric (like F1 score) that the tuning method can optimize.


**Reasoning**:
Define the train_and_evaluate function and then iterate through the hyperparameter grid to train and evaluate the model for each combination, storing the results. Finally, identify and print the best hyperparameters based on the F1 score.



In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import numpy as np

# Ensure necessary libraries are imported and available
# Already imported: pandas, numpy, sklearn.model_selection.train_test_split,
# sklearn.preprocessing.LabelEncoder, sklearn.metrics.classification_report,
# sklearn.metrics.f1_score, torch, transformers, torch.optim.AdamW, transformers.get_scheduler, tqdm


# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Encode the labels (already done in previous cell, but good to be explicit if needed)
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['Genre'])

# Train-test split (already done in previous cell)
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df['Description'].tolist(),
#     df['label'].tolist(),
#     test_size=0.2,
#     random_state=42,
#     stratify=df['label']
# )

# Tokenization using RoBERTa tokenizer (already done in previous cell)
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# PyTorch Dataset (already defined and created instances in previous cell)
# class MovieDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.labels)
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

# train_dataset = MovieDataset(train_encodings, train_labels)
# val_dataset = MovieDataset(val_encodings, val_labels)


# 1. Define a train_and_evaluate Function
def train_and_evaluate(learning_rate, batch_size, num_epochs, train_dataset, val_dataset, num_labels):
    """
    Trains and evaluates a RoBERTa model for sequence classification.

    Args:
        learning_rate (float): Learning rate for the optimizer.
        batch_size (int): Batch size for data loaders.
        num_epochs (int): Number of training epochs.
        train_dataset (Dataset): PyTorch Dataset for training.
        val_dataset (Dataset): PyTorch Dataset for validation.
        num_labels (int): Number of output labels.

    Returns:
        float: Macro F1 score on the validation set.
    """
    # Initialize the RoBERTa model
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
    model.to(device)

    # Set up the optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Set up the scheduler
    num_training_steps = num_epochs * len(train_loader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1} (LR: {learning_rate}, BS: {batch_size})")
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item())

    # Evaluation loop
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    # Calculate macro F1 score
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    # Clear GPU memory
    del model
    torch.cuda.empty_cache()

    return macro_f1

# Assuming param_grid, train_dataset, val_dataset, and label_encoder are available from previous cells
# If not, uncomment and run the necessary previous cells

# 2. Integrate with GridSearchCV (Manual Iteration)
results = []
num_labels = len(label_encoder.classes_) # Get the number of labels

# Iterate through all parameter combinations
for lr in param_grid['learning_rate']:
    for bs in param_grid['per_device_train_batch_size']:
        for epochs in param_grid['num_train_epochs']:
            print(f"Training with LR: {lr}, Batch Size: {bs}, Epochs: {epochs}")
            # Call the train_and_evaluate function
            f1 = train_and_evaluate(lr, bs, epochs, train_dataset, val_dataset, num_labels)
            results.append({
                'learning_rate': lr,
                'batch_size': bs,
                'num_epochs': epochs,
                'f1_score': f1
            })
            print(f"Finished training with LR: {lr}, Batch Size: {bs}, Epochs: {epochs}. F1 Score: {f1:.4f}")
            print("-" * 50)


# 3. Identify the Best Parameters
best_f1 = -1
best_params = None

for res in results:
    if res['f1_score'] > best_f1:
        best_f1 = res['f1_score']
        best_params = {k: v for k, v in res.items() if k != 'f1_score'}

print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Macro F1 Score: {best_f1:.4f}")


Using device: cuda
Training with LR: 1e-05, Batch Size: 8, Epochs: 2


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 (LR: 1e-05, BS: 8):   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 2 (LR: 1e-05, BS: 8):   0%|          | 0/500 [00:00<?, ?it/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 (LR: 1e-05, BS: 8):   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 2 (LR: 1e-05, BS: 8):   0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 